In [2]:
import numpy as np

In [120]:
def price_european_call_explicit(S0, K, T, r, sigma, Smax, M, N):
    dt = T / M
    dS = Smax / N
    S = np.linspace(0, Smax, N+1)
    
    V = np.maximum(S - K, 0)
    
    alpha = 0.5 * dt * ((sigma**2) * (np.arange(N+1)**2) - r * np.arange(N+1))
    beta  = 1 - dt * ((sigma**2) * (np.arange(N+1)**2) + r)
    gamma = 0.5 * dt * ((sigma**2) * (np.arange(N+1)**2) + r * np.arange(N+1))

    for n in range(M):
        Vn = V.copy()
        for i in range(1, N):
            V[i] = alpha[i] * Vn[i-1] + beta[i] * Vn[i] + gamma[i] * Vn[i+1]
        
        V[0] = 0
        V[-1] = Smax - K * np.exp(-r * dt * (M - n - 1))
    
    return np.interp(S0, S, V)

price_european_call_explicit(S0=100, K=100, T=1, r=0.07, sigma=0.25, Smax=400, N=100, M=10000)

13.332586028515205

In [114]:
def price_european_call_implicit(S0, K, T, r, sigma, Smax, M, N):
    dt = T / M
    dS = Smax / N
    S = np.linspace(0, Smax, N+1)
    
    V = np.maximum(S - K, 0)

    i = np.arange(1, N)
    alpha = 0.5 * dt * (sigma**2 * i**2 - r * i)
    beta  = 1 + dt * (sigma**2 * i**2 + r)
    gamma = 0.5 * dt * (sigma**2 * i**2 + r * i)

    a = -alpha
    b = beta
    c = -gamma

    for _ in range(M):
        d = V[1:N]

        d[0]   += alpha[0] * V[0]
        d[-1] += gamma[-1] * (Smax - K * np.exp(-r * dt))

        c_prime = np.zeros(N-1)
        d_prime = np.zeros(N-1)

        c_prime[0] = c[0] / b[0]
        d_prime[0] = d[0] / b[0]

        for j in range(1, N-1):
            denom = b[j] - a[j] * c_prime[j-1]
            c_prime[j] = c[j] / denom
            d_prime[j] = (d[j] - a[j] * d_prime[j-1]) / denom

        V_new = np.zeros(N-1)
        V_new[-1] = d_prime[-1]
        for j in reversed(range(N-2)):
            V_new[j] = d_prime[j] - c_prime[j] * V_new[j+1]

        V[1:N] = V_new
        V[0] = 0
        V[N] = Smax - K * np.exp(-r * (_ + 1) * dt)

    return np.interp(S0, S, V)

price_european_call_implicit(S0=100, K=100, T=1, r=0.07, sigma=0.25, Smax=400, M=10000, N=100)

13.360266497976534

In [128]:
import numpy as np

def price_european_call_crank_nicolson(S0, K, T, r, sigma, Smax, M, N):
    dt = T / M
    dS = Smax / N
    S = np.linspace(0, Smax, N+1)  # asset price grid

    V = np.maximum(S - K, 0.0)     # payoff at maturity

    i = np.arange(1, N)
    alpha = 0.5 * dt * (sigma**2 * i**2 - r * i)
    beta  =        dt * (sigma**2 * i**2 + r)
    gamma = 0.5 * dt * (sigma**2 * i**2 + r * i)

    a_CN = -0.5 * alpha
    b_CN =  1.0 + 0.5 * beta
    c_CN = -0.5 * gamma

    a_BE = -alpha
    b_BE = 1.0 + beta
    c_BE = -gamma

    rannacher_steps = 2  # number of backward Euler steps at the start

    for k in range(M):
        t_n = (k + 1) * dt
        Vn1 = V.copy()

        # set boundaries for the current time step
        V[0] = 0.0
        V[-1] = Smax - K * np.exp(-r * t_n)
        VN_n = V[-1]

        # build RHS vector d
        if k < rannacher_steps:
            # Backward Euler step (first 2 steps)
            d = Vn1[1:N].copy()
            d[-1] += gamma[-1] * VN_n
            a = a_BE
            b = b_BE
            c = c_BE
        else:
            # Crank-Nicolson
            d = 0.5 * alpha * Vn1[0:N-1] + (1.0 - 0.5 * beta) * Vn1[1:N] + 0.5 * gamma * Vn1[2:N+1]
            d[-1] += 0.5 * gamma[-1] * VN_n
            a = a_CN
            b = b_CN
            c = c_CN

        # Thomas algorithm
        c_prime = np.empty(N-1); d_prime = np.empty(N-1)
        c_prime[0] = c[0] / b[0]
        d_prime[0] = d[0] / b[0]
        for j in range(1, N-1):
            denom = b[j] - a[j] * c_prime[j-1]
            c_prime[j] = c[j] / denom
            d_prime[j] = (d[j] - a[j] * d_prime[j-1]) / denom

        x = np.empty(N-1)
        x[-1] = d_prime[-1]
        for j in range(N-3, -1, -1):
            x[j] = d_prime[j] - c_prime[j] * x[j+1]

        # update V for the next step
        V[1:N] = x
        V[0]   = 0.0
        V[-1]  = Smax - K * np.exp(-r * ((k+1) * dt))

    return np.interp(S0, S, V)

price_european_call_crank_nicolson(S0=100, K=100, T=1.0, r=0.07, sigma=0.25, Smax=400, M=10000, N=100)

13.332447722456694

In [78]:
S0=100
K=100
T=1.0
r=0.07
sigma=0.25
Smax=400
M=800
N=100
S = np.linspace(0, Smax, N+1) # generates N+1 numbers from 0 to Smax (space indices)
V = np.maximum(S - K, 0.0)

In [80]:
V

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   4.,   8.,  12.,  16.,  20.,  24.,  28.,
        32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,  64.,  68.,  72.,
        76.,  80.,  84.,  88.,  92.,  96., 100., 104., 108., 112., 116.,
       120., 124., 128., 132., 136., 140., 144., 148., 152., 156., 160.,
       164., 168., 172., 176., 180., 184., 188., 192., 196., 200., 204.,
       208., 212., 216., 220., 224., 228., 232., 236., 240., 244., 248.,
       252., 256., 260., 264., 268., 272., 276., 280., 284., 288., 292.,
       296., 300.])